# Doğrusal Regresyon Egzersizleri

50 adet Startup'ın araştırma ve geliştirmeye yönelik harcaması, yönetime yönelik harcaması, pazarlama harcaması, kazandıkları kar miktarı ve kuruldukları lokasyon bilgisi bulunmaktadır. Amaç kar miktarını tahmin etmektir. Bu bir sayısal tahmin problemidir ve bağımlı değişkenimiz "Profit".

Numpy, matplotlib.pyplot, pandas ve seaborn kütüphanelerini çekirdeğe dahil edelim.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

Dizinde bulunan veri çerçevemizi startups değişkenine atayalım. startups değişkenini df değişkenine kopyalayarak kullanmaya başlayalım.

In [ ]:
startups = pd.read_csv("../input/50-startups/50_Startups.csv", sep=",")
df = startups.copy()

İlk 5 gözlemini yazdıralım.

In [ ]:
df.head() 

Veri çerçevesinin bilgilerini görüntüleyelim.

In [ ]:
df.info()

Kaç gözlem ve öznitelikten oluştuğunu görüntüleyelim.

In [ ]:
df.shape

→ 50 gözlem ve 5 değişkenden oluşuyor.

Eksik verileri kontrol edelim.

In [ ]:
df.isna().sum() # görmüş olduğumuz gibi eksik verimiz yok

Korelasyon matrisi çizdirelim.

In [ ]:
corr = df.corr()
corr

→ Korelasyon 1.0'e yaklaştıkça değerler arasındaki ilişki o kadar güçlenir. Burada **Profit** ile **R&D Spend** arasında 1'e yakın bir değer (0.972900) gözlenmiş bu demek oluyor ki ikisi arasında pozitif nerdeyse mükemmel bir ilişki vardır. **Marketing Spend** ile de **Administration** arasında da negatif bir ilişki olduğunu söyleyebiliriz.

Seaborn ile korelasyon matrisinin ısı haritasını çizdirelim.

In [ ]:
sns.heatmap(corr,
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values);

R&D Spend ve Profit arasındaki korelasyonu daha iyi görebilmek için scatterplot çizdirelim.

In [ ]:
sns.scatterplot(x = "R&D Spend", y = "Profit", data = df); 

In [ ]:
sns.regplot(df["R&D Spend"],df["Profit"] , ci=None)

* profit sadece bir değişkene bağlı değil birden fazla feature olduğundan → multiple 
* multiple: bir y eksenine birden fazla şeyin etki etmesi demektir
* y = b0+ b1.x1+ b2.x2 doğru denklemi
* profit= b0+ b1.r&d +b2.administration+....
* b0: constant/sabit/bias ~ y eksenini kestiği nokta
* b1: coeff/katsayı ~eğim
* amaç: min(MSE)
* profit: bağımlı değişken~dependent variable
* administration,r&d,Marketing Spend: bağımsız değişken~independent variable



Sayısal değişkenlerin dağılımını görmek için df üzerinden histogram çizdirelim.

In [ ]:
df.hist(figsize =(15,20))

→Histogram şemaları verilerin devam eden ya da belirli zaman aralığındaki dağılımını göstermek için kullanılır. Değerlerin nerede yoğunlaştığını, maksimum ve minimum noktalarını, veriler arasındaki kopmalar ve olağandışı değerler olup olmadığı hakkında tahminde bulunulmasına yardımcı olur.

* R&D Spend histogramını bakarsak; verilerin 0'dan 150 bin daha fazla tahmini olarak 170 bin gibi değerlere sahip olduğunu görebiliriz aynı zamanda verimizin 50bin ile 80 bin arasında daha yoğun olduğunu görüyoruz. Burada en küçük değer olarak sıfırı görüyoruz. Yani bazı startuplar araştırma geliştirmeye(Research and Development) bütçe ayırmadığı ya da çok küçük bir bütçe ayırdığını görebiliyoruz.

* Marketing Spend histogramını bakarsak; verilerin 0 ve tahmini olarak 320 bin arasında daha yoğun olduğunu söyleyebiliriz. Yani bazı startuplar pazarlamaya hiç bütçe ayırmazken bazı startuplar ise çok büyük bütçeler ayırabiliyor.

* Administration histogramını bakarsak; verilerin 90 bin ile 150 bin arasında daha yoğun olduğunu görebiliyoruz. Genel bir gözle bakarsak 100 bin  ile 200 bin aralığında bütçeden yapılan harcamalarsa en fazla harcama yönetime yapılmış.Yönetim bu parayı şirket karı için kullanmış olabilirler bu da profit'e yansır ya da sadece kendi giderleri için kullanmış olabilirler. Bu yüzden yönetim harcamaları ile karın korelasyonuna bakmak istiyorum.

In [ ]:
df.corr()["Administration"]["Profit"]

* Administrationı ve Profit arasındaki korelasyona baktığımda "0.20071656826872125" değerini gördük. Yönetim bu parayı şirket karı için kullanmamış diyebiliriz. Belki bu harcamanın içinde maaşlar yer alıyor olabilir.

Veri çerçevesinin temel istatistik değerlerini görüntüleyelim.

In [ ]:
df.describe().T

* Standart sapma çok yüksek neredeyse özniteliklerimin ortalama değerlerinin yarısı kadar.




Eğer bir veri ortalamaya yakın ise standart sapma düşük olur ama eğer ortalamaya uzakta bir dağılım söz konusuysa standart sapma değeri  büyük olur. Varyans standart sapmanın karesi olduğundan ortalamaları ve standart sapma değerleri ile varyans yorumu yapabiliriz.
* Standart sapması en yüksek olan değer "Marketing Spend" dir. Bu yüzden en yüksek varyansa sahiptir.
* Standart sapması en düşük değer ise "Administration" dir. Bu yüzden en küçük varyans değerine sahiptir.

State'a ait benzersiz değerleri görüntüleyelim.

In [ ]:
df["State"].unique()

get_dummies yardımıyla State'a dair kategorik öznitelik çıkarımlarında bulunalım. Çünkü State'ların birbirine üstünlüğü yok, nominaller. Ordinal değil.

In [ ]:
df['State'] = pd.Categorical(df['State'])
dfDummies = pd.get_dummies(df['State'], prefix = 'StateOf') 
#pd.get_dummies(),veri işleme için kullanılır. Kategorik verileri kukla veya gösterge değişkenlere dönüştürür.
#prefix = 'StateOf' , sütun adlarına ön ek eklemek için kullanılır
dfDummies.head()#50 değeri göstermeyip sadece ilk beş gözlemi getiriyorum

In [ ]:
df = pd.concat([df, dfDummies], axis = 1)
df.head()
#concat fonksiyonunu kullanırken sütunların isimlerini ayrıca belirtmemize gerek olmadan serileri birleştirebiliriz.

State özniteliğini silip dummy olarak yaratılan State'lardan da birisini hariç tutarak veri çerçevemizi güncelleyelim.

In [ ]:
df.drop(["State", "StateOf_New York"], axis = 1, inplace = True)

In [ ]:
df.head()

* Modelde yer alan nitel vasıflı değişkenlere kukla değişken denir. Kukla değişkenler genel olarak ikil/binom (kadın,erkek) ve çoklu (mezuniyet: ilköğretim, ortaöğretim, lise, lisans, lisansüstü vb) biçimde iki yaklaşımla ölçülür.Binom ölçeklendirmede nitel değişken 1 ve 0 durumunu alır(kadın=1, erkek=0 gibi). Çoklu (polychotomous) ölçeklendirmede ise değişkene ait niteliksel vasıflar uygun ve aralarında anlamsal bir farklılığı yansıtacak bir biçimde sıralanır( ilköğretim =0, ortaöğretim=1, lise=2, lisans=3, lisansüstü=4 gibi).

* Modelde eğer nitel bir değişkenin ikili yapıda tanımlarını kullanacak isek, model sağlıklı bir biçimde tahmin edilemez çünkü katsayılardan oluşan matrisinin tersi alınamaz. **Modelin sağlıklı bir biçimde tahmin edilebilmesi için genelde kontrol değişkeni olarak tanımlanan kukla değişkenini modelden düşürmemiz gerekir. Yani sizin araştırmadaki amacınız kadın olma vasfının bağımlı değişken üzerindeki etkisi is tabloda verilen "cinsiyet" ve "erkek" sütunlarını kullanmamamız gerekir.

→ State özniteliğini ve statelerden birini seçip siliyorum.


Veri çerçevemizi bağımlı ve bağımsız değişkenler olmak üzere bölütleyelim.

In [ ]:
X = df.drop("Profit", axis = 1)
y = df["Profit"]

Bağımlı ve bağımsız değişkenleri kontrol edelim.

In [ ]:
X.head(10) #bagimsiz değişkenleri görüyorum

In [ ]:
y.head(10) #bağımlı değişken olan profiti görüntülüyorum

→ Profit özniteliği 0 dan 10. gözleme gidildikçe azalmaktadır. Bu bağımsız değişkenlerden kaynaklanıyor.
* Lineer regresyon yaptığımızda gözlem sırası karışacak, karışık şekilde öğrenmesini sağlayacağız yani önce karı artıran sonra azaltan değerleri bir arada öğrenmeyecek.

Bu bağımlı ve bağımsız değişkenlerden train ve test olmak üzere 4 parça oluşturalım. Bunu yapmak için train_test_split kullanalım.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

#Test size'ı %20 , Train(öğrenme) ise %80 olarak kullanıyorum burada öğrenme veri kümesini küçük tutarak sonuçların çok iyi olmasını engelliyorum.
#Veri setimizdeki test size 10 veriye, train ise 40 veriye tekabül ediyor.
#Random state belirtmek bu noktada önemli belirtilmezse, kodu her çalıştırdığımızda yeni bir rastgele değer oluşturulur.
#train ve test veri kümelerinin her seferinde farklı değerleri olur. Python bu data'yı her seferinde farklı yerlerinden böler.
#Eğer bir random_state değeri belirlersek her seferinde o değere göre böler. Yani aynı test verileriyle test etmiş oluruz.
#Random state kullanarak rassallığı engelleriz.

4 parça değişkeni kontrol edelim.

In [ ]:
X_train.head(10)

In [ ]:
X_test #test olduğundan 10 veri geldi (%20 )

In [ ]:
y_train.head(10)

In [ ]:
y_test #Test olduğu için bu da 10 değer geldi.

LinearRegression'u çekirdeğe dahil edip modeli inşa edelim.

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

Modeli eğitmek için bağımlı bağımsız değişkenlerden oluşturulan eğitim verilerini verelim.

In [ ]:
model = lr.fit(X_train, y_train)

Modele daha önce görmediği bağımlı test değişkenini tahmin ettirelim. Bu tahmin değerlerimizi y_pred değişkenine atayalım.

In [ ]:
y_pred = model.predict([[90000, 65000, 100000, 0, 0]])
y_pred 

* 90 bin research and development'a, 65 bin yönetim'e, 100 bin pazarlama'ya harcayan ve New York'lu olan startuplar 118664 dolar kar elde edecektir diye bir çıkarımında bulunabilirim. Burada New Yorku seçtim California = 0 ve Florida = 0 ise New York = 1 dir. Sadece iki değerin durumuna göre üçüncü durumu anlayabiliyoruz.

Tahminleri ve gerçek değerleri bir veri çerçevesinde toplayıp üzerinde göz gezdirelim.

In [ ]:
df["kar_tahmin"] = model.predict(X)
df.head(10)

sklearn bünyesinde barınan metrics'i çekirdeğe dahil edelim ve MAE, MSE, RMSE değerlerini görüntüleyelim.

In [ ]:
from sklearn.metrics import mean_absolute_error
MAE = mean_absolute_error(df["Profit"], df["kar_tahmin"])
print("Hata Mutlak Ortalama Degeri(MAE):", MAE) 

In [ ]:
from sklearn.metrics import mean_squared_error
MSE = mean_squared_error(df["Profit"], df["kar_tahmin"])
print("Hata Kareler Ortalaması Degeri(MSE):", MSE)  

In [ ]:
import math
RMSE = math.sqrt(MSE)
print("Hata Kareler Ortalamasının Karekök Degeri(RMSE):", RMSE) 

Modelin R Squared değerini eğitim verileri üzerinden yazdıralım.

In [ ]:
print("R Squared:", model.score(X_train,y_train))

* R Squared değeri modeldeki bağımsız değişkenlerin bağımlı değişkenleri ifade etmesidir. bu değer 1'e ne kadar yakınsa o kadar iyidir bizim modelimide bu 0.95018 çıktı 1'e yakın bir değer gayet iyi diyebiliriz.

Dileyenler statsmodel kullanarak hangi özniteliklerin model için %95 güvenilirlikle ne kadar anlamlı olup olmadığına da bakabilir. Modelde bazı feature selection işlemleri yaparak tekrardan eğitip yeni sonuçlar mukayese edilebilir.

In [ ]:
import statsmodels.api as sm
stmodel = sm.OLS(y, X).fit()

**OLS(Sıradan En Küçük Kareler Yöntemi)** doğrusal regresyon modeli kurulurken verilerin ortasından geçen eğime (çizgiye) en az kare farkı ile yaklaşmaya çalışan bir yöntemdir.

In [ ]:
stmodel.summary()

***R^2 ifadesi:*** Bağımsız değişkenlerin bağımlı değişkendeki değişikliği açıklama yüzdesidir.
* 0.988 oldukça yüksek bir rakam. R^2 ne kadar değişken eklenirse o kadar şişmektedir. Şişmenin önüne geçmek için düzeltilmiş R^2 değeri ortaya çıkar. Daha sağlıklı bir açıklanabilirlik oranı vermektedir.
***F-statistic:*** Modelin anlamlılığını test etmek için kullanılan istatistiktir.
***Prob(F- statistic:*** Modelin anlamlı olup olmadığını anlamak için kullanılır.
* Prob(F- statistic 0.05 den oldukça küçük bir değere sahip. Bu yüzden modelin anlamlı olduğunu söyleyebiliriz. 
***coef:*** Bağımsız değişkenlerin katsayılarını ifade eder. (b1, b2, b3 katsayılarını ifade ediyor.) 
***std err:** Bağımsız değişkenlerin standart hata değeridir. Ne kadar küçükse, doğruluk seviyesi o kadar büyük olur.
***t istatistiği*** anlamlılığı ifade eder. 
***P>t modellemek için kullandığımız bütün değişkenler anlamlıdır.**
* 0.05 ten küçük değerler elde ettiğimiz sürece anlamlı değişkenler olur. 

* Burada önemli bir nokta ise State özniteliklerine baktığımızda P değerinin 0.05'ten büyük değerleri olduğunu görüyoruz. Bu model için iyi bir şey değil. Bnları sileceğim buna ek oalrak kar tahminini de sileceğim ki eski değerlerle işlem yapıp aynı sonuçları bulmayayım.

In [ ]:
df.head()

In [ ]:
df.drop(["StateOf_California", "StateOf_Florida", "kar_tahmin"], axis = 1, inplace = True)

In [ ]:
df.head()

In [ ]:
X1 = df.drop("Profit", axis = 1)
X1.head() 
#X'e yeni tablomuzu aktarıyorum

In [ ]:
y.head() # y'de yani profitte bir değişiklik yapmadık zaten.

In [ ]:
X1_train, X1_test, y_train, y_test = train_test_split(X1, y, test_size = 0.2, random_state = 0)

#Test size'ı %20 , Train(öğrenme) ise %80 olarak kullanıyorum burada öğrenme veri kümesini küçük tutarak sonuçların çok iyi olmasını engelliyorum.
#Veri setimizdeki test size 10 veriye, train ise 40 veriye tekabül ediyor.
#random state belirtmek bu noktada önemli belirtilmezse, kodu her çalıştırdığımızda yeni bir rastgele değer oluşturulur.
#train ve test veri kümelerinin her seferinde farklı değerleri olur. Python bu datayı her seferinde farklı yerlerinden böler.
#Eğer bir random_state değeri belirlersek her seferinde o değere göre böler. Yani aynı test verileriyle test etmiş oluruz. Rassallığ engelleriz.

In [ ]:
lm = LinearRegression()
model2 = lm.fit(X1_train, y_train)

In [ ]:
y_pred2 = model2.predict([[90000, 65000, 100000]])
y_pred2

In [ ]:
df["kar_tahmin"] = model2.predict(X1)
df.head(10)

* Değerler eskisinden farklı. MSE RMSE ve R Squared değerlerini d bulup yeni modelin daha iyi mi yoksa daha kötü mü olduğuna bakalım.

In [ ]:
MSE1 = mean_squared_error(df["Profit"], df["kar_tahmin"])
print("Hata Kareler Ortalaması Degeri(MSE1):", MSE1)  

* Hata Kareler Ortalaması Degeri(MSE): 81957374.24681288 idi.
* Yeni  MSE1 değeri 81056127.50873542 çok küçük de olsa bir düşüş var. Tahmin değerleri ile gerçek değerler arasında daha iyi bir regresyon olduğunu söyleyebiliriz.


In [ ]:
RMSE1 = math.sqrt(MSE1)
print("Hata Kareler Ortalamasının Karekök Degeri(RMSE1):", RMSE1)

Hata Kareler Ortalamasının Karekök Degeri(RMSE): 9053.031218703098 idi yeni RMSE1 değeri 9003.117654942393

In [ ]:
model2.score(X1_train,y_train) 

R Squared: 0.9501847627493607 idi r squared değerinin ise çok küçük bir düşüş yaşadığını görüyoruz.